## 1 - SetUp Environment

In [1]:
import numpy as np
import pandas as pd
import pickle

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2 - Load Dataframe

now we should load the dataframe that we saved in the previous section.

In [2]:
with open('/content/drive/MyDrive/Python/Regression/Assets/df(1.2).pickle', 'rb') as file:
    df = pickle.load(file)

df.head(3)

,Start Date,Duration,Cost,Team Member,Height,Frequency,Signal Strength,Antenna Type,Orientation,Power Supply,Zone
0,3/4/2019,241.0,516773.0,12.0,24.0,Very Low Frequencies (VLF),3,Dielectric,Omni-directional,Solar-powered,North
1,10/2/2019,608.0,954888.0,22.0,42.0,Very Low Frequencies (VLF),3,Dielectric,Circular2,Active,Center
4,6/29/2019,772.0,932640.0,14.0,43.0,Very High Frequencies (VHF),3,NaN,Horizontal,Active,Center


## 3 - Declare Variable

In this section, I work on just date variables and I try to clean and preprocess them. The date variables of this dataset are:</br>
<li> Start Date

In [3]:
date_list = ['Start Date']

## 4 - Data Type Conversion

since, the date variable may have different format, first we should check its type and convert it to date format.

In [4]:
import datetime as dt

for i in range(len(date_list)):
    df[date_list[i]] = pd.to_datetime(df[date_list[i]] , format = '%m/%d/%Y').dt.date
    
df.head(2)

,Start Date,Duration,Cost,Team Member,Height,Frequency,Signal Strength,Antenna Type,Orientation,Power Supply,Zone
0,2019-03-04,241.0,516773.0,12.0,24.0,Very Low Frequencies (VLF),3,Dielectric,Omni-directional,Solar-powered,North
1,2019-10-02,608.0,954888.0,22.0,42.0,Very Low Frequencies (VLF),3,Dielectric,Circular2,Active,Center


## 5 - Handle Null Values

then we can check whether this variable has null values or not.

In [5]:
df = df.reset_index(drop=True)

def null_checker():
    for i in range(len(date_list)):
        print(date_list[i] , ':' , df.loc[df[date_list[i]].isnull()].index)

In [6]:
null_checker()

Start Date : Int64Index([8, 43, 65, 82, 114, 171, 174, 185, 188, 190, 193, 215, 220, 238,
            253, 283, 298],
           dtype='int64')


the result shows we have null value. then we must make sure about the rule of 5%.

In [7]:
for i in range(len(date_list)):
    count_nan = df[date_list[i]].isna().sum()
    print(date_list[i] , ':' , round((count_nan / df.shape[0]) * 100, 2) , '%')

Start Date : 5.47 %


Since it is more than 5%, we should impute this variable. I use the combination of one past and one coming date values.

In [8]:
for i in range(len(date_list)):
    df[date_list[i]] = df[date_list[i]].bfill().ffill()

now we check the statuse of null value.

In [9]:
null_checker()

Start Date : Int64Index([], dtype='int64')


## 6 - Sanity Check

in this section we must make sure there is a logical relationship between the start date and the duration. since, this data is about the before end of 2022, there must not be a combination of start date and duration that exceeds from ebd of 2022

In [10]:
df.head(2)

,Start Date,Duration,Cost,Team Member,Height,Frequency,Signal Strength,Antenna Type,Orientation,Power Supply,Zone
0,2019-03-04,241.0,516773.0,12.0,24.0,Very Low Frequencies (VLF),3,Dielectric,Omni-directional,Solar-powered,North
1,2019-10-02,608.0,954888.0,22.0,42.0,Very Low Frequencies (VLF),3,Dielectric,Circular2,Active,Center


In [11]:
from datetime import datetime, timedelta

Finish_Date = []

for i in range(len(df)):
    duration_value = df.iloc[i,1]
    x = df.iloc[i,0] + timedelta(days = duration_value) # for start date
    Finish_Date.append(x)
    
df['Finish Date'] = Finish_Date
df['Finish Date'] = pd.to_datetime(df['Finish Date'])
df.head(3)

,Start Date,Duration,Cost,Team Member,Height,Frequency,Signal Strength,Antenna Type,Orientation,Power Supply,Zone,Finish Date
0,2019-03-04,241.0,516773.0,12.0,24.0,Very Low Frequencies (VLF),3,Dielectric,Omni-directional,Solar-powered,North,2019-10-31
1,2019-10-02,608.0,954888.0,22.0,42.0,Very Low Frequencies (VLF),3,Dielectric,Circular2,Active,Center,2021-06-01
2,2019-06-29,772.0,932640.0,14.0,43.0,Very High Frequencies (VHF),3,NaN,Horizontal,Active,Center,2021-08-09


then we can check the finish date with date of end of 2022.

In [12]:
df['Check Date'] = df['Finish Date'] > datetime(2022, 12, 31)
df['Check Date'].sum()

1

it shows one row has a date more than end of 2022. we should check that if it was imputed rows (handling null value) or not.

In [13]:
print(df['Check Date'].idxmax())

283


let's see the finish date:

In [14]:
print(df.iloc[283,0] + timedelta(days = df.iloc[283,1]))

2024-05-31


it is about the index 283 that we imputed. so, we have to consult with experts to find the real date. after consulting, we realized that the start date is 2018-03-12 (before it was null and we had to impute it). so we need to change the value:

In [15]:
new_date_string = '2018-03-12'
new_date = datetime.strptime(new_date_string, '%Y-%m-%d').date()
df.iloc[283, 0] = new_date
pd.DataFrame(df.iloc[283,:])

,283
Start Date,2018-03-12
Duration,1439.0
Cost,1841891.0
Team Member,10.0
Height,35.0
Frequency,Very High Frequencies (VHF)
Signal Strength,0
Antenna Type,Printed Circuit Board (PCB)
Orientation,Vertical
Power Supply,Active


Also, we can remove finish date and check columns

In [16]:
df = df.drop(['Finish Date','Check Date'],axis=1)
df.head(2)

,Start Date,Duration,Cost,Team Member,Height,Frequency,Signal Strength,Antenna Type,Orientation,Power Supply,Zone
0,2019-03-04,241.0,516773.0,12.0,24.0,Very Low Frequencies (VLF),3,Dielectric,Omni-directional,Solar-powered,North
1,2019-10-02,608.0,954888.0,22.0,42.0,Very Low Frequencies (VLF),3,Dielectric,Circular2,Active,Center


## 7 - Extract Year, Month, Weekdays

In this step, we should extract year, month and days of week for every single date and put them into new variables.

In [17]:
Year = []
Month = []

def day_to_week(date_value):
    return date_value.weekday()

for i in range(df.shape[0]):
    Year.append(df['Start Date'][i].year)
    Month.append(df['Start Date'][i].month)
    
df['Year'] = Year
df['Month'] = Month
df['Week Day'] = df['Start Date'].apply(day_to_week)

in week day column, the 0 means monday and 6 means sunday.then we can remove the Start Date column

In [18]:
df = df.drop('Start Date',axis=1)

Finnaly, we should change the order.

In [19]:
order = [
    'Year',
    'Month',
    'Week Day',
    'Duration', 
    'Cost',
    'Team Member', 
    'Height',
    'Frequency', 
    'Signal Strength',
    'Antenna Type', 
    'Orientation', 
    'Power Supply', 
    'Zone']

df = df[order]
df.head(3)

,Year,Month,Week Day,Duration,Cost,Team Member,Height,Frequency,Signal Strength,Antenna Type,Orientation,Power Supply,Zone
0,2019,3,0,241.0,516773.0,12.0,24.0,Very Low Frequencies (VLF),3,Dielectric,Omni-directional,Solar-powered,North
1,2019,10,2,608.0,954888.0,22.0,42.0,Very Low Frequencies (VLF),3,Dielectric,Circular2,Active,Center
2,2019,6,5,772.0,932640.0,14.0,43.0,Very High Frequencies (VHF),3,NaN,Horizontal,Active,Center


## Check Point

In [20]:
with open('/content/drive/MyDrive/Python/Regression/Assets/df(1.3).pickle', 'wb') as file:
    pickle.dump(df, file)